In [1]:
import yahist
import awkward as ak
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
def gaus(x, *p0):
    N, mu, sigma = p0
    return N*np.exp(-(x-mu)**2/(2.0*sigma**2))


### Gameplan

1. Get distrobutions of rising and falling clock edges as well as arams clock
    - also get toa, trigger distrobution for completeness
2. Show dts for all 3 scenarios
3. Look at jitter for all 3 scenarios

In [2]:
# old_clock = "/home/users/hswanson13/tbanalysis/output_analysis_data/original_clock/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/fullCollection.parquet"
# new_clock_frst_rising_edg = "/home/users/hswanson13/tbanalysis/output_analysis_data/custom_my_clock/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/fullCollection.parquet"
# new_clock_second_rising_edg = "/home/users/hswanson13/tbanalysis/output_analysis_data/second_rising/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/fullCollection.parquet"
import sys
import json
import pickle as pk
import os

def get_data(data_path):
    _, file_ext = os.path.splitext(data_path)
    # data_path = join(data_dir, data_file)
    with open(data_path, "rb") as f:
        if file_ext == '.json':
            data = ak.Array(json.load(f))
        elif file_ext == '.pkl':
            data = pk.load(f)
        else:
            raise NotImplementedError('This file extension is not implemented yet')
    return data



def plot_gauss_fit(ax, hist_counts, bin_centers, p0, color='red', lw=2):
    # Do a simple gaussian fit
    try:
        coeff, var_martix = curve_fit(gaus, bin_centers, hist_counts, p0=p0)
    except RuntimeError:
        coeff = p0
    ax.plot(
        bin_centers,
        gaus(bin_centers, *coeff),
        color=color,
        label='Gaussian Fit\n mean: {:.2f} \n sigma: {:.3f}'.format(coeff[1],abs(coeff[2])),
        linewidth=lw
    )

dt_frst_edg = get_data("/home/users/hswanson13/tbanalysis/output_analysis_data/nearest_0_edge/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/dt_first_edge_data.pkl")
dt_scnd_edg = get_data("/home/users/hswanson13/tbanalysis/output_analysis_data/nearest_0_edge/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/dt_second_edge_data.pkl")
dt_desy = get_data("/home/users/hswanson13/tbanalysis/output_analysis_data/nearest_0_edge/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/dt_tot_raw_data.pkl")
dt_nearest_zero = get_data("/home/users/hswanson13/tbanalysis/output_analysis_data/nearest_0_edge/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/dt_nearest_zero_data.pkl")

row, col = 9, 8
fig = plt.figure(constrained_layout=True,  figsize=(14, 6))
ax = plt.subplot(1,1,1)

frst_vals = np.array(dt_frst_edg[row][col]["dt_first"])
scnd_vals = np.array(dt_scnd_edg[row][col]["dt_second"])
desy_vals = np.array(dt_desy[row][col]["dt"])
near_zero_vals = np.array(dt_nearest_zero[row][col]["dt_near_0"])

dt_frst_hist = yahist.Hist1D(frst_vals)
dt_scnd_hist = yahist.Hist1D(scnd_vals)
dt_desy_hist = yahist.Hist1D(desy_vals)
dt_zero_hist = yahist.Hist1D(near_zero_vals)

dt_frst_hist.plot(ax, color='blue', label="First Edge")
dt_scnd_hist.plot(ax, color='red', label='Second Edge')
dt_desy_hist.plot(ax, color='orange', label="Original DESY")
dt_zero_hist.plot(ax, color='black', label="Nearest 0")
    
plot_gauss_fit(ax, dt_desy_hist.counts, dt_desy_hist.bin_centers, [np.max(dt_desy_hist.counts), np.mean(dt_desy_hist.bin_centers), 0.1], lw=4)
plot_gauss_fit(ax, dt_zero_hist.counts, dt_zero_hist.bin_centers, [np.max(dt_zero_hist.counts), np.mean(dt_zero_hist.bin_centers), 0.1], color='blue')

ax.legend(loc=1, prop={'size': 18}, frameon=True,fancybox=True)

ax.set_ylabel("counts")
ax.set_xlabel("t (ns)")
ax.set_title("dt From Clocks Timestamp")

# df = ak.from_parquet(new_clock_second_rising_edg)

# print(df)

# dt_hist = yahist.Hist1D(ak.flatten(df), bins=np.linspace(-25,25,650))

# fig, ax = plt.subplots()
# ax.set_yscale('log')
# ax.set_xlabel("time (ns)")
# ax.set_title("dt DESY (First Rising Edge)")
# dt_hist.plot(ax)

FileNotFoundError: [Errno 2] No such file or directory: '/home/users/hswanson13/tbanalysis/output_analysis_data/nearest_0_edge/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/dt_first_edge_data.pkl'